<a href="https://colab.research.google.com/github/IverMartinsen/deeplearning/blob/master/problem3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title import modules { form-width: "20%" }
import sys
from google.colab import drive

drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/DL home exam/Problem 3/problem3/')

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape
from tensorflow.keras import Model
import numpy as np
from deshuffler import deshuffler
from load_data import load_data

In [ ]:
#@title load data { form-width: "20%" }
x_tr, y_tr, x_te, y_te = load_data(
    '/content/gdrive/MyDrive/DL home exam/Problem 3/problem3/DataShuffled.npz')

x_tr = np.moveaxis(x_tr, 1, -1) / 255
y_tr = np.moveaxis(y_tr, 1, -1)
x_te = np.moveaxis(x_te, 1, -1) / 255
y_te = np.moveaxis(y_te, 1, -1)

In [ ]:
#@title define models { form-width: "20%" }
class CNN(Model):
    def __init__(self):
        super().__init__()

        self.conv1 = Conv2D(64, 3, padding='same', activation='relu')
        self.norm1 = BatchNormalization()
        self.pool1 = MaxPool2D()
        
        self.conv2 = Conv2D(128, 3, padding='same', activation='relu')
        self.norm2 = BatchNormalization()
        self.pool2 = MaxPool2D()
        
        self.conv3 = Conv2D(256, 3, padding='same', activation='relu')
        self.norm3 = BatchNormalization()
        self.conv4 = Conv2D(256, 3, padding='same', activation='relu')
        self.norm4 = BatchNormalization()
        self.pool3 = MaxPool2D()
        
        self.conv5 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm5 = BatchNormalization()
        self.conv6 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm6 = BatchNormalization()
        self.pool4 = MaxPool2D()
        
        self.conv7 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm7 = BatchNormalization()
        self.conv8 = Conv2D(512, 3, padding='same', activation='relu')
        self.norm8 = BatchNormalization()
        self.pool5 = MaxPool2D()
        
        self.flatten = Flatten()

        self.dense1 = Dense(4096,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2())
        self.dense2 = Dense(4096,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2())
        self.dense3 = Dense(1000,
                            activation='relu',
                            kernel_regularizer=tf.keras.regularizers.L2())

    def call(self, images):
        x = self.pool1(self.norm1(self.conv1(images)))

        x = self.pool2(self.norm2(self.conv2(x)))

        x = self.pool3(self.norm4(self.conv4(self.norm3(self.conv3(x)))))

        x = self.pool4(self.norm6(self.conv6(self.norm5(self.conv5(x)))))

        x = self.pool5(self.norm8(self.conv8(self.norm7(self.conv7(x)))))

        return self.dense3(self.dense2(self.dense1(self.flatten(x))))    

class Puzzle_solver(Model):
    def __init__(self):
        super().__init__()

        self.dense = Dense(16, activation='softmax')
        self.reshape = Reshape((4, 4))
        self.cnn = CNN()

    def call(self, images):
        input = tf.concat((self.cnn(images[:, :48, :48, :]),
                           self.cnn(images[:, :48, 48:, :]),
                           self.cnn(images[:, 48:, :48, :]),
                           self.cnn(images[:, 48:, 48:, :])), axis = 1)
        
        return self.reshape(self.dense(input))
    
    def solve(self, images):
        return np.argmax(self.predict(images), axis = 2)

    def accuracy(self, images, labels):
        iscorrect = np.prod(self.solve(images) == labels, axis = 1)

        acc = {}
        acc['image'] = float(np.sum(iscorrect) / iscorrect.shape)
        acc['quarter'] = float(np.sum(self.solve(images) == labels) / iscorrect.shape) / 4

        return acc

In [ ]:
#@title train model { form-width: "20%" }
model = Puzzle_solver()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy())

model.load_weights('/content/gdrive/MyDrive/DL home exam/Problem 3/ckpt_ps')
model.cnn.load_weights('/content/gdrive/MyDrive/DL home exam/Problem 3/ckpt_cn')

callbacks = [tf.keras.callbacks.EarlyStopping(patience=50,
                                              restore_best_weights=True)]

history = model.fit(x_tr,
                    y_tr,
                    epochs=50,
                    validation_split=0.2,
                    callbacks=callbacks,
                    shuffle=True)

model.save_weights('/content/gdrive/MyDrive/DL home exam/Problem 3/ckpt_ps',
                   overwrite=True)
model.cnn.save_weights('/content/gdrive/MyDrive/DL home exam/Problem 3/ckpt_cn',
                       overwrite=True)